In [4]:
import sys
sys.path.append('/home/megagatlingpea/workdir/protenix4science')
import json
import os
import numpy as np
import biotite.structure as struc
from biotite.structure.io import load_structure

from protenix.data.constants import mmcif_restype_3to1 as AA_3TO1
from protenix.data.constants import PROTEIN_BACKBONE_ATOM_NAMES, NA_BACKBONE_ATOM_NAMES, BACKBONE_ATOM_NAMES, PROTEIN_REP_RESIDUE, RNA_REP_RESIDUE, DNA_REP_RESIDUE

from protenix.data.utils import convert_bb_configs,convert_metadata

In [5]:
output_json = convert_bb_configs('./../examples/bb_generation.json',dump=False)
#metadata = convert_metadata(output_json.pop('metadata'))

In [6]:
output_json

[{'sequences': [{'proteinChain': {'sequence': 'GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGEGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGHGGGGGGGGGGGGGGGGGGGGGGGGGGKGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG',
     'count': 1}},
   {'proteinChain': {'sequence': 'GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGEMALLDGQPRSADGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGHGSEMGGGGGGGGGGGGGGGGGGGGGGGGGGKGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG',
     'count': 1}}],
  'metadata': [{'fixed_atom': {'A86': ['O', 'C', 'CA'],
     'A168': ['NE2', 'CD2', 'CE1', 'CG', 'ND1'],
     'A197': ['NZ', 'CE', 'CD']},
    'fixed': ['47-47',
     'A86-86',
     '56-56',
     'A168-168',
     '26-26',
     'A197-197',
     '48-48']},
   {'fixed_atom': {},
    'fixed': ['47-47',
     'B86-98',
     '56-56',
     'B168-172',
     '26-26',
     'B197-197',
     '48-48']}],
  'name': '7pzb'}]